### Import cw-simulate module and create new instance

In [ ]:
import { SimulateCosmWasmClient } from "@oraichain/cw-simulate";

senderAddress = "orai14vcw5qk0tdvknpa38wz46js5g7vrvut8lk0lk6";
client = new SimulateCosmWasmClient({
  chainId: "Oraichain",
  bech32Prefix: "orai"
});


### Setup account balance

In [ ]:
import { coin } from "@cosmjs/stargate";

client.app.bank.setBalance(senderAddress, [coin('1000000000', 'orai'), coin('1000000000', 'usdt')]);

console.log(client.app.bank.getBalance(senderAddress));

### Deploy required contracts for simple case

In [ ]:
import { toBinary } from "@cosmjs/cosmwasm-stargate";
import { DaoDaoCoreClient } from "@oraichain/dao-contracts-sdk";

const { codeId } = await client.upload(senderAddress, new Uint8Array(await fetch("/wasm/dao-dao-core.wasm").then(res=>res.arrayBuffer())), 'auto');
const { codeId: cw20Id } = await client.upload(senderAddress, new Uint8Array(await fetch("/wasm/cw20-base.wasm").then(res=>res.arrayBuffer())), 'auto');
const { codeId: voteModId } = await client.upload(senderAddress, new Uint8Array(await fetch("/wasm/dao-voting-cw20-staked.wasm").then(res=>res.arrayBuffer())), 'auto');
const { codeId: cw20StakeId } = await client.upload(senderAddress, new Uint8Array(await fetch("/wasm/cw20-stake.wasm").then(res=>res.arrayBuffer())), 'auto');
const { codeId: proposalSingleId } = await client.upload(senderAddress, new Uint8Array(await fetch("/wasm/dao-proposal-single.wasm").then(res=>res.arrayBuffer())), 'auto');

const { contractAddress } = await client.instantiate(senderAddress, codeId, {
    name: 'DAO DAO',
    description: 'A DAO that builds DAOs.',
    automatically_add_cw20s: true,
    automatically_add_cw721s: true,
    voting_module_instantiate_info: {
        code_id: voteModId,
        msg: toBinary({
            token_info: {
                new: {
                    code_id: cw20Id,
                    label: 'DAO DAO governance token',
                    name: 'DAO',
                    symbol: 'DAO',
                    decimals: 6,
                    initial_balances: [{ amount: '100000000', address: senderAddress }],
                    staking_code_id: cw20StakeId
                }
            },
            active_threshold: {
                absolute_count: {
                    count: '100'
                }
            }
        }),
        admin: { core_module: {} },
        label: 'voting module'
    },
    proposal_modules_instantiate_info: [{
        code_id: proposalSingleId,
        msg: toBinary({
            threshold: {
                threshold_quorum: {
                    quorum: { percent: '0.015' },
                    threshold: { majority: {} }
                }
            },
            max_voting_period: { time: 604800 }, // One week.
            only_members_execute: true,
            allow_revoting: false,
            pre_propose_info: {
                anyone_may_propose: {}
            },
            close_proposal_on_execution_failure: true
        }),
        admin: { core_module: {} },
        label: 'governance module'
    }]
}, "dao-dao-core", "auto");
  
daoContract = new DaoDaoCoreClient(client, senderAddress, contractAddress);

### Create proposal

In [ ]:
import { Ok } from "ts-results";
import { toBinary } from "@cosmjs/cosmwasm-stargate";
import { DaoVotingCw20StakedClient, DaoProposalSingleClient, Cw20StakeClient } from "@oraichain/dao-contracts-sdk";

const votingModuleContract = new DaoVotingCw20StakedClient(client, senderAddress, await daoContract.votingModule());
const govTokenAddress = await votingModuleContract.tokenContract();
const proposalModules = await daoContract.proposalModules({});    
const stakingContract = new Cw20StakeClient(client, senderAddress, await votingModuleContract.stakingContract());
const proposalModulesAddr = proposalModules.find((m) => m.status === 'enabled').address;

proposalModuleContract = new DaoProposalSingleClient(client, senderAddress, proposalModulesAddr);

// deposit token
await client.execute(
    senderAddress,
    govTokenAddress,
    {
        send: {
            contract: stakingContract.contractAddress,
            amount: '100',
            msg: toBinary({
                stake: {}
            })
        }
    },
    'auto'
);


client.app.store.tx((setter) => {
    setter('height')(client.app.height + 1);
    setter('time')(client.app.time + 5 * 1e9);
    return Ok(true);
});


// make_proposal
await proposalModuleContract.propose({
    title: "title",
    description: "description",
    msgs: []
});

### Unstake some tokens to make it inactive

In [ ]:
await stakingContract.unstake({
    amount: '50'
});

try {
    await proposalModuleContract.propose({
        title: 'title',
        description: 'description',
        msgs: []
    });    
} catch (ex) {
    // expect: the DAO is currently inactive, you cannot create proposals
    console.log(ex.message);
}

### Vote the proposal

In [ ]:
await proposalModuleContract.vote({
    proposalId: 1,
    vote: 'yes'
});

const { proposal } = await proposalModuleContract.proposal({ proposalId: 1 });

// proposal is passed
console.log(proposal);